<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Flight-Delay" data-toc-modified-id="Flight-Delay-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Flight Delay</a></span><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Ознакомление-с-данными" data-toc-modified-id="Ознакомление-с-данными-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Ознакомление с данными</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Найти-аэропорт-с-минимальной-задержкой-вылета" data-toc-modified-id="Найти-аэропорт-с-минимальной-задержкой-вылета-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Найти аэропорт с минимальной задержкой вылета</a></span></li><li><span><a href="#Самая-пунктуальная-авиакомпания-на-прилет-в-Los-Angeles-International-Airport" data-toc-modified-id="Самая-пунктуальная-авиакомпания-на-прилет-в-Los-Angeles-International-Airport-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Самая пунктуальная авиакомпания на прилет в Los Angeles International Airport</a></span></li><li><span><a href="#Найти-аэропорт,-где-самолёты-проводят-больше-всего-времени-на-рулении-(среднее-значение)" data-toc-modified-id="Найти-аэропорт,-где-самолёты-проводят-больше-всего-времени-на-рулении-(среднее-значение)-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Найти аэропорт, где самолёты проводят больше всего времени на рулении (среднее значение)</a></span></li><li><span><a href="#Построить-модель-которая-будет-выбирать-топ-3-аэропорта-прилета-(вероятность-опоздания-минимальная-–-RMSE-метрика),-в-зависимости-от-аэропорта-вылета" data-toc-modified-id="Построить-модель-которая-будет-выбирать-топ-3-аэропорта-прилета-(вероятность-опоздания-минимальная-–-RMSE-метрика),-в-зависимости-от-аэропорта-вылета-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Построить модель которая будет выбирать топ 3 аэропорта прилета (вероятность опоздания минимальная – RMSE метрика), в зависимости от аэропорта вылета</a></span></li></ul></li></ul></div>

# Flight Delay

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

In [2]:
airports = pd.read_csv('airports.csv')
airlines = pd.read_csv('airlines.csv')
flights = pd.read_csv('flights.csv', dtype={'ORIGIN_AIRPORT': str,'DESTINATION_AIRPORT': str } )

In [3]:
airlines.tail()

,IATA_CODE,AIRLINE
9,DL,Delta Air Lines Inc.
10,EV,Atlantic Southeast Airlines
11,HA,Hawaiian Airlines Inc.
12,MQ,American Eagle Airlines Inc.
13,VX,Virgin America


In [4]:
airports.tail()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
317,WRG,Wrangell Airport,Wrangell,AK,USA,56.48433,-132.36982
318,WYS,Westerly State Airport,West Yellowstone,MT,USA,44.68840,-111.11764
319,XNA,Northwest Arkansas Regional Airport,Fayetteville/Springdale/Rogers,AR,USA,36.28187,-94.30681
320,YAK,Yakutat Airport,Yakutat,AK,USA,59.50336,-139.66023
321,YUM,Yuma International Airport,Yuma,AZ,USA,32.65658,-114.60597


In [5]:
flights.tail()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
5819074,2015,12,31,4,B6,688,N657JB,LAX,BOS,2359,...,753.0,-26.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5819075,2015,12,31,4,B6,745,N828JB,JFK,PSE,2359,...,430.0,-16.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5819076,2015,12,31,4,B6,1503,N913JB,JFK,SJU,2359,...,432.0,-8.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5819077,2015,12,31,4,B6,333,N527JB,MCO,SJU,2359,...,330.0,-10.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5819078,2015,12,31,4,B6,839,N534JB,JFK,BQN,2359,...,442.0,2.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


## Ознакомление с данными

In [6]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 31 columns):
 #   Column               Dtype  
---  ------               -----  
 0   YEAR                 int64  
 1   MONTH                int64  
 2   DAY                  int64  
 3   DAY_OF_WEEK          int64  
 4   AIRLINE              object 
 5   FLIGHT_NUMBER        int64  
 6   TAIL_NUMBER          object 
 7   ORIGIN_AIRPORT       object 
 8   DESTINATION_AIRPORT  object 
 9   SCHEDULED_DEPARTURE  int64  
 10  DEPARTURE_TIME       float64
 11  DEPARTURE_DELAY      float64
 12  TAXI_OUT             float64
 13  WHEELS_OFF           float64
 14  SCHEDULED_TIME       float64
 15  ELAPSED_TIME         float64
 16  AIR_TIME             float64
 17  DISTANCE             int64  
 18  WHEELS_ON            float64
 19  TAXI_IN              float64
 20  SCHEDULED_ARRIVAL    int64  
 21  ARRIVAL_TIME         float64
 22  ARRIVAL_DELAY        float64
 23  DIVERTED             int64  
 24

In [7]:
airlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   IATA_CODE  14 non-null     object
 1   AIRLINE    14 non-null     object
dtypes: object(2)
memory usage: 352.0+ bytes


In [8]:
airports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   IATA_CODE  322 non-null    object 
 1   AIRPORT    322 non-null    object 
 2   CITY       322 non-null    object 
 3   STATE      322 non-null    object 
 4   COUNTRY    322 non-null    object 
 5   LATITUDE   319 non-null    float64
 6   LONGITUDE  319 non-null    float64
dtypes: float64(2), object(5)
memory usage: 17.7+ KB


In [9]:
flights.describe()

,YEAR,MONTH,DAY,DAY_OF_WEEK,FLIGHT_NUMBER,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,...,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
count,5819079.0,5.819079e+06,5.819079e+06,5.819079e+06,5.819079e+06,5.819079e+06,5.732926e+06,5.732926e+06,5.730032e+06,5.730032e+06,...,5.819079e+06,5.726566e+06,5.714008e+06,5.819079e+06,5.819079e+06,1.063439e+06,1.063439e+06,1.063439e+06,1.063439e+06,1.063439e+06
mean,2015.0,6.524085e+00,1.570459e+01,3.926941e+00,2.173093e+03,1.329602e+03,1.335204e+03,9.370158e+00,1.607166e+01,1.357171e+03,...,1.493808e+03,1.476491e+03,4.407057e+00,2.609863e-03,1.544643e-02,1.348057e+01,7.615387e-02,1.896955e+01,2.347284e+01,2.915290e+00
std,0.0,3.405137e+00,8.783425e+00,1.988845e+00,1.757064e+03,4.837518e+02,4.964233e+02,3.708094e+01,8.895574e+00,4.980094e+02,...,5.071647e+02,5.263197e+02,3.927130e+01,5.102012e-02,1.233201e-01,2.800368e+01,2.143460e+00,4.816164e+01,4.319702e+01,2.043334e+01
min,2015.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-8.200000e+01,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,-8.700000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2015.0,4.000000e+00,8.000000e+00,2.000000e+00,7.300000e+02,9.170000e+02,9.210000e+02,-5.000000e+00,1.100000e+01,9.350000e+02,...,1.110000e+03,1.059000e+03,-1.300000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2015.0,7.000000e+00,1.600000e+01,4.000000e+00,1.690000e+03,1.325000e+03,1.330000e+03,-2.000000e+00,1.400000e+01,1.343000e+03,...,1.520000e+03,1.512000e+03,-5.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,2.000000e+00,3.000000e+00,0.000000e+00
75%,2015.0,9.000000e+00,2.300000e+01,6.000000e+00,3.230000e+03,1.730000e+03,1.740000e+03,7.000000e+00,1.900000e+01,1.754000e+03,...,1.918000e+03,1.917000e+03,8.000000e+00,0.000000e+00,0.000000e+00,1.800000e+01,0.000000e+00,1.900000e+01,2.900000e+01,0.000000e+00
max,2015.0,1.200000e+01,3.100000e+01,7.000000e+00,9.855000e+03,2.359000e+03,2.400000e+03,1.988000e+03,2.250000e+02,2.400000e+03,...,2.400000e+03,2.400000e+03,1.971000e+03,1.000000e+00,1.000000e+00,1.134000e+03,5.730000e+02,1.971000e+03,1.331000e+03,1.211000e+03


## Подготовка данных

In [10]:
#проверка на наличие дубликатов

In [11]:
flights.duplicated().sum()

0

In [12]:
airports.duplicated().sum()

0

In [13]:
airlines.duplicated().sum()

0

In [14]:
#проверка на наличие пропусков
flights.isna().sum()

YEAR                         0
MONTH                        0
DAY                          0
DAY_OF_WEEK                  0
AIRLINE                      0
FLIGHT_NUMBER                0
TAIL_NUMBER              14721
ORIGIN_AIRPORT               0
DESTINATION_AIRPORT          0
SCHEDULED_DEPARTURE          0
DEPARTURE_TIME           86153
DEPARTURE_DELAY          86153
TAXI_OUT                 89047
WHEELS_OFF               89047
SCHEDULED_TIME               6
ELAPSED_TIME            105071
AIR_TIME                105071
DISTANCE                     0
WHEELS_ON                92513
TAXI_IN                  92513
SCHEDULED_ARRIVAL            0
ARRIVAL_TIME             92513
ARRIVAL_DELAY           105071
DIVERTED                     0
CANCELLED                    0
CANCELLATION_REASON    5729195
AIR_SYSTEM_DELAY       4755640
SECURITY_DELAY         4755640
AIRLINE_DELAY          4755640
LATE_AIRCRAFT_DELAY    4755640
WEATHER_DELAY          4755640
dtype: int64

In [15]:
airlines.isna().sum()

IATA_CODE    0
AIRLINE      0
dtype: int64

In [16]:
airports.isna().sum()

IATA_CODE    0
AIRPORT      0
CITY         0
STATE        0
COUNTRY      0
LATITUDE     3
LONGITUDE    3
dtype: int64

In [17]:
flights_NULL = flights.isnull().sum()*100/flights.shape[0]
flights_NULL

YEAR                    0.000000
MONTH                   0.000000
DAY                     0.000000
DAY_OF_WEEK             0.000000
AIRLINE                 0.000000
FLIGHT_NUMBER           0.000000
TAIL_NUMBER             0.252978
ORIGIN_AIRPORT          0.000000
DESTINATION_AIRPORT     0.000000
SCHEDULED_DEPARTURE     0.000000
DEPARTURE_TIME          1.480526
DEPARTURE_DELAY         1.480526
TAXI_OUT                1.530259
WHEELS_OFF              1.530259
SCHEDULED_TIME          0.000103
ELAPSED_TIME            1.805629
AIR_TIME                1.805629
DISTANCE                0.000000
WHEELS_ON               1.589822
TAXI_IN                 1.589822
SCHEDULED_ARRIVAL       0.000000
ARRIVAL_TIME            1.589822
ARRIVAL_DELAY           1.805629
DIVERTED                0.000000
CANCELLED               0.000000
CANCELLATION_REASON    98.455357
AIR_SYSTEM_DELAY       81.724960
SECURITY_DELAY         81.724960
AIRLINE_DELAY          81.724960
LATE_AIRCRAFT_DELAY    81.724960
WEATHER_DE

In [18]:
#строки с nan
airports[airports.isna().any(axis=1)]

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
96,ECP,Northwest Florida Beaches International Airport,Panama City,FL,USA,NaN,NaN
234,PBG,Plattsburgh International Airport,Plattsburgh,NY,USA,NaN,NaN
313,UST,Northeast Florida Regional Airport (St. August...,St. Augustine,FL,USA,NaN,NaN


In [19]:
flights['dDate'] = pd.to_datetime(flights[['YEAR','MONTH','DAY']], dayfirst=True)

In [20]:
#объединяем таблицы

flights = flights.merge(airports, left_on='ORIGIN_AIRPORT', right_on='IATA_CODE', how='inner')
flights = flights.merge(airports, left_on='DESTINATION_AIRPORT', right_on='IATA_CODE', how='inner')

In [21]:
flights = flights.merge(airlines, left_on='AIRLINE', right_on='IATA_CODE', how='inner')
flights = flights.drop(['AIRLINE_x','IATA_CODE'], axis=1)
flights = flights.rename(columns={"AIRLINE_y":"AIRLINE"})

In [22]:
flights = flights.drop(['LATITUDE_x',
                        'LONGITUDE_x',
                        'STATE_y',
                        'COUNTRY_y',
                        'LATITUDE_y',
                        'LONGITUDE_y','STATE_x',
                        'COUNTRY_x'], axis=1)

In [23]:
flights = flights.rename(columns={'IATA_CODE_x':'Orig_Airport_Code',
                                  'AIRPORT_x':'Orig_Airport_Name',
                                  'CITY_x':'Origin_city',
                                  'IATA_CODE_y':'Dest_Airport_Code',
                                  'AIRPORT_y':'Dest_Airport_Name',
                                  'CITY_y':'Destination_city'})

In [24]:
flights.columns.unique()

Index(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'FLIGHT_NUMBER', 'TAIL_NUMBER',
       'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE',
       'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF',
       'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'WHEELS_ON',
       'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY',
       'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY',
       'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY',
       'WEATHER_DELAY', 'dDate', 'Orig_Airport_Code', 'Orig_Airport_Name',
       'Origin_city', 'Dest_Airport_Code', 'Dest_Airport_Name',
       'Destination_city', 'AIRLINE'],
      dtype='object')

In [25]:
flights.ORIGIN_AIRPORT.unique()

array(['ANC', 'LAX', 'SFO', 'LAS', 'DEN', 'SLC', 'PDX', 'FAI', 'MSP',
       'PHX', 'ORD', 'GEG', 'HNL', 'ONT', 'MCO', 'BOS', 'DFW', 'IAH',
       'BNA', 'PHL', 'IAD', 'JFK', 'SMF', 'AUS', 'MCI', 'ATL', 'DCA',
       'SAT', 'CHS', 'FLL', 'TPA', 'EWR', 'OAK', 'ABQ', 'BWI', 'MSY',
       'RDU', 'DTW', 'TUS', 'PSP', 'STL', 'OGG', 'SJC', 'KOA', 'SAN',
       'LIH', 'BLI', 'SNA', 'BUR', 'JNU', 'KTN', 'SIT', 'SEA', 'OME',
       'BRW', 'PSG', 'YAK', 'GST', 'OTZ', 'SCC', 'BET', 'ADK', 'CDV',
       'ADQ', 'DLG', 'AKN', 'WRG', 'CVG', 'SJU', 'MKE', 'BOI', 'SBN',
       'BUF', 'JAX', 'MSN', 'IND', 'CLE', 'BDL', 'OMA', 'SYR', 'ROC',
       'XNA', 'RSW', 'RNO', 'PIT', 'MIA', 'LGA', 'OKC', 'ATW', 'MEM',
       'CLT', 'GRB', 'FAR', 'MSO', 'RAP', 'DLH', 'FSD', 'CMH', 'PSC',
       'MOT', 'BIS', 'BZN', 'MDW', 'DSM', 'ALB', 'GRR', 'DAY', 'TVC',
       'JAC', 'ICT', 'FCA', 'EGE', 'BIL', 'FNT', 'VPS', 'SGF', 'PWM',
       'GSP', 'RIC', 'GSO', 'PBI', 'PVD', 'TUL', 'LIT', 'PNS', 'TYS',
       'MHT', 'SAV',

## Найти аэропорт с минимальной задержкой вылета 

In [26]:
a = flights.groupby(['ORIGIN_AIRPORT','Origin_city']).mean()['DEPARTURE_DELAY'].sort_values()

In [27]:
a[:15]

ORIGIN_AIRPORT  Origin_city
YAK             Yakutat       -6.292237
CNY             Moab          -6.058537
EKO             Elko          -3.771760
VEL             Vernal        -3.740000
CDV             Cordova       -3.261868
LWS             Lewiston      -2.976190
PIH             Pocatello     -1.328221
TWF             Twin Falls    -0.895911
ITO             Hilo          -0.863947
BRD             Brainerd      -0.065292
BLI             Bellingham    -0.060649
SCC             Deadhorse      0.154122
ADQ             Kodiak         0.405034
LIH             Lihue          0.556254
DLG             Dillingham     0.558442
Name: DEPARTURE_DELAY, dtype: float64

In [28]:
b = flights[flights['DEPARTURE_DELAY'] > 0]

In [29]:
b1 = b.groupby(['ORIGIN_AIRPORT','Origin_city']).mean()['DEPARTURE_DELAY'].sort_values()

In [30]:
b1[:15]

ORIGIN_AIRPORT  Origin_city     
AKN             King Salmon         13.800000
ITO             Hilo                16.411424
DLG             Dillingham          20.960000
HNL             Honolulu            22.167616
OGG             Kahului             22.508665
KOA             Kailua/Kona         22.888927
LIH             Lihue               22.952563
CNY             Moab                23.909091
WYS             West Yellowstone    24.243902
ADQ             Kodiak              24.543307
SEA             Seattle             25.178592
SNA             Santa Ana           25.322597
OAK             Oakland             25.361167
DAL             Dallas              25.480517
SMF             Sacramento          26.741141
Name: DEPARTURE_DELAY, dtype: float64

## Самая пунктуальная авиакомпания на прилет в Los Angeles International Airport

In [31]:
LA_arrival = flights[flights['Dest_Airport_Name']=='Los Angeles International Airport']

In [32]:
# если не считать 5-минутное опоздание за опоздание

LA_5 = LA_arrival[LA_arrival['ARRIVAL_DELAY']<=5] 
LA_p = ((LA_5.groupby('AIRLINE').size()/
         LA_arrival.groupby('AIRLINE').size())*100).sort_values(ascending=False).reset_index()
LA_p = LA_p .rename(columns={0: 'Accuracy'})
LA_p[:5]

,AIRLINE,Accuracy
0,Hawaiian Airlines Inc.,78.933847
1,Alaska Airlines Inc.,74.148396
2,Delta Air Lines Inc.,71.841729
3,American Airlines Inc.,70.160330
4,JetBlue Airways,69.352483


## Найти аэропорт, где самолёты проводят больше всего времени на рулении (среднее значение)

In [33]:
taxing = flights.copy()

In [34]:
taxing['taxing'] = taxing['TAXI_OUT'] + taxing['TAXI_IN']

In [35]:
t1 = taxing.groupby(['Orig_Airport_Name','Origin_city']).mean()['taxing'].sort_values(ascending=False)

In [36]:
t1[:10]

Orig_Airport_Name                                                       Origin_city 
LaGuardia Airport (Marine Air Terminal)                                 New York        34.541541
Sioux Gateway Airport                                                   Sioux City      34.387479
Sawyer International Airport                                            Marquette       34.209459
John F. Kennedy International Airport (New York International Airport)  New York        34.201981
Waterloo Regional Airport                                               Waterloo        31.706186
Ithaca Tompkins Regional Airport                                        Ithaca          30.266667
St. Cloud Regional Airport                                              St Cloud        30.166667
Philadelphia International Airport                                      Philadelphia    29.667411
Eagle County Regional Airport                                           Eagle           29.076475
Sloulin Field International Airpo

## Построить модель которая будет выбирать топ 3 аэропорта прилета (вероятность опоздания минимальная – RMSE метрика), в зависимости от аэропорта вылета  

In [37]:
#создаем функцию для генерации фичей

def make_features(data, max_lag, rolling_mean_size):
    data['year'] = data.dDate.dt.year
    data['month'] = data.dDate.dt.month
    data['day'] = data.dDate.dt.day
    data['dayofweek'] = data.dDate.dt.dayofweek
    for lag in range(1, max_lag + 1):
        data['lag_{}'.format(lag)] = data['ARRIVAL_DELAY'].shift(lag)

In [38]:
flight_grouped = flights.groupby(['dDate','DESTINATION_AIRPORT'])['ARRIVAL_DELAY'].sum().reset_index()
flight_grouped['dDate'] = pd.to_datetime(flight_grouped['dDate'])

In [39]:
where_to_go = []

for dest in flight_grouped.DESTINATION_AIRPORT.unique():
    tempo = flight_grouped[flight_grouped.DESTINATION_AIRPORT==dest][['dDate','ARRIVAL_DELAY']]
    tempo.columns = ['dDate','ARRIVAL_DELAY']
    tempo = pd.DataFrame(tempo)
    
    try:
    
        make_features(tempo,21,7)
        tempo.dropna(inplace=True)
        tempo.set_index('dDate',inplace=True)
           
        X_train,X_test,y_train, y_test = train_test_split(tempo.drop('ARRIVAL_DELAY', axis=1),tempo.ARRIVAL_DELAY, shuffle=False, test_size=0.25)

        model_lr = LinearRegression()
        model_lr.fit(X_train,y_train)

        y_predicted_lr = model_lr.predict(X_test)
        where_to_go.append([dest,y_test.mean(),np.sqrt(mean_squared_error(y_test, y_predicted_lr))])

    except Exception as e:
        print('Error', str(e))

In [40]:
where_to_go = pd.DataFrame(where_to_go)
where_to_go.columns = ['DESTINATION_AIRPORT','MEAN_ARRIVAL_DELAY_IN_PAST','RMSE']

In [41]:
start_airport = np.random.choice(flights['ORIGIN_AIRPORT'].unique()) 
where_to_go_from_start_airport = flights[flights.ORIGIN_AIRPORT==start_airport]['DESTINATION_AIRPORT'].unique()

where_to_go_from_start_airport = pd.DataFrame(where_to_go_from_start_airport)
where_to_go_from_start_airport.columns = ['DESTINATION_AIRPORT']

top3 = where_to_go_from_start_airport\
    .merge(where_to_go,on='DESTINATION_AIRPORT',how='inner')\
    .sort_values(by=['RMSE','MEAN_ARRIVAL_DELAY_IN_PAST'],ascending=[True, True])\
    .head(3)

city_from = airports[airports.IATA_CODE==start_airport]['CITY']

print('Лучшие направления с аэропорта', start_airport, city_from)
print('')
print(top3)

Лучшие направления с аэропорта FWA 118    Fort Wayne
Name: CITY, dtype: object

  DESTINATION_AIRPORT  MEAN_ARRIVAL_DELAY_IN_PAST         RMSE
3                 DTW                 -150.860759  2400.825391
0                 MSP                  547.721519  2985.553102
4                 DFW                  -12.607595  6782.533363


In [42]:
top_un = top3.reset_index()
go_to_city = top_un.merge(airports, left_on='DESTINATION_AIRPORT', right_on='IATA_CODE', how='inner')
go_to_city[['CITY','AIRPORT']]

,CITY,AIRPORT
0,Detroit,Detroit Metropolitan Airport
1,Minneapolis,Minneapolis-Saint Paul International Airport
2,Dallas-Fort Worth,Dallas/Fort Worth International Airport
